# 🚀 **Hafta 3: Transformers Temelleri ve Optimizasyon**

<div style="background:#eaf6fb; border-left:6px solid #2980B9; padding:16px; border-radius:8px;">
<span style="font-size:1.2em; font-weight:bold; color:#2980B9;">Bu notlar, <span style="color:#C0392B;">Hugging Face Transformers dünyasını, temel kavramları ve üretim kalitesinde NLP için optimizasyon tekniklerini </span><span style="color:#229954;">adım adım</span> öğretmek için hazırlanmıştır.</span>
</div>

---

## 📅 İçindekiler
1. [Transformers Nedir?](#transformers-nedir)
2. [AutoTokenizer & AutoModel: Temel Kavramlar](#autotokenizer-automodel)
3. [Pipeline: Tek Satırda NLP](#pipeline)
4. [Transformer Mimarileri: GPT, BERT, T5](#mimariler)
5. [Cihaz ve Model Optimizasyonu](#optimizasyon)
6. [Performans Ölçümü ve Benchmarking](#benchmark)
7. [En İyi Uygulamalar](#bestpractices)
8. [Ek Kaynaklar](#resources)

<div style="background:#f9e79f; border-left:6px solid #b7950b; padding:10px; border-radius:8px;">
🔔 <b>İpucu:</b> <span style="color:#b7950b;">Her bölümde önemli noktalar <b>kalın</b>, renkli ve ikonlarla vurgulanmıştır!</span>
</div>

---

## 1. Transformers Nedir?

**Transformers**, günümüzün en güçlü doğal dil işleme (NLP) modellerinin temelini oluşturan bir mimaridir. 2017'de "Attention is All You Need" makalesiyle tanıtılmıştır.

**Avantajları:**
- Uzun metinlerde bağlamı çok iyi anlar.
- Paralel işlemeye uygundur (hızlıdır).
- Farklı görevler için kolayca uyarlanabilir.

**Hugging Face Transformers** kütüphanesi, bu mimarinin binlerce hazır modelini kolayca kullanmanızı sağlar.

## 2. AutoTokenizer & AutoModel: Temel Kavramlar

<div style="background:#f4f8fb; border-left:6px solid #229954; padding:12px; border-radius:8px;">
🔑 <b style="color:#229954;">AutoTokenizer</b> ve <b style="color:#2980B9;">AutoModel</b> <b>birlikte</b> veya <b>ayrı ayrı</b> kullanılabilir. Her birinin rolü farklıdır ve birlikte kullanıldığında <span style="color:#CA6F1E;">tam bir inference pipeline</span> kurarsınız.
</div>

### 🔹 <span style="color:#229954;">AutoTokenizer</span> Nedir?
- <b>Görev:</b> Metni, modelin anlayacağı <b style="color:#CA6F1E;">tokenlara</b> (sayılara) çevirir.
- <b>Örnek:</b> "Merhaba dünya" → [101, 1234, 4567, 102]
- <b>Ekstra:</b> Tokenizer, <b>decode</b> işlemiyle sayıları tekrar metne çevirebilir.

### 🔹 <span style="color:#2980B9;">AutoModel</span> Nedir?
- <b>Görev:</b> Tokenları alır, <b style="color:#8E44AD;">modelin matematiksel katmanlarından</b> geçirir ve çıktı üretir.
- <b>Çıktı:</b> Genellikle <b>hidden states</b> (gizli temsiller) veya görev özelinde başka çıktılar.

<div style="background:#fadbd8; border-left:6px solid #C0392B; padding:10px; border-radius:8px;">
🧩 <b>Farkları:</b><br>
<b style="color:#229954;">AutoTokenizer</b> sadece <b>metin ↔ token</b> dönüşümü yapar.<br>
<b style="color:#2980B9;">AutoModel</b> ise <b>token ↔ çıktı</b> dönüşümünü sağlar.<br>
<b>Birlikte kullanınca:</b> <span style="color:#CA6F1E;">Ham metinden model çıktısına tam yolculuk</span> elde edilir!
</div>

### 👩‍💻 <span style="color:#229954;">AutoTokenizer</span> ve <span style="color:#2980B9;">AutoModel</span> ile Adım Adım

<div style="background:#e8daef; border-left:6px solid #8E44AD; padding:10px; border-radius:8px;">
📝 <b>Flashcard:</b> <b>AutoTokenizer</b> olmadan <b>AutoModel</b> kullanırsanız, modele <b>doğrudan metin</b> veremezsiniz! Model <b>sayısal token</b> ister.
</div>

In [ ]:
# 1️⃣ Metni tokenlara çevir (encode)
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer("Transformers harika!", return_tensors="pt")
print(inputs)

# 2️⃣ Tokenları modele ver, çıktı al
model = AutoModel.from_pretrained("bert-base-uncased")
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)  # (batch_size, sequence_length, hidden_size)

# 3️⃣ (İsteğe bağlı) Tokenları tekrar metne çevir (decode)
decoded = tokenizer.decode(inputs['input_ids'][0])
print(decoded)

<div style="background:#f9e79f; border-left:6px solid #b7950b; padding:10px; border-radius:8px;">
💡 <b>Önemli:</b> <b>AutoTokenizer</b> ve <b>AutoModel</b> <span style="color:#C0392B;">her zaman aynı model adıyla</span> çağrılmalı! (örn. "bert-base-uncased")
</div>

<details>
<summary><b>🎓 Sıkça Sorulan: Sadece AutoTokenizer veya sadece AutoModel kullanılır mı?</b></summary>
<ul>
<li><b>Sadece AutoTokenizer:</b> Metni tokenlara çevirip başka bir yerde (ör. veri ön işleme) kullanabilirsiniz.</li>
<li><b>Sadece AutoModel:</b> Tokenları başka bir tokenizer ile hazırladıysanız veya ileri düzey kullanımda mümkündür.</li>
<li><b>En yaygın ve güvenli yol:</b> <b>İkisini birlikte</b> kullanmak!</li>
</ul>
</details>

<div style="background:#f4fbf4; border-left:6px solid #229954; padding:10px; border-radius:8px;">
🟢 <b>Özet:</b> <b>AutoTokenizer</b> = <span style="color:#CA6F1E;">metin → token</span> <br>
<b>AutoModel</b> = <span style="color:#CA6F1E;">token → çıktı</span>
</div>

## 3. Pipeline: Tek Satırda NLP

<div style="background:#f4f8fb; border-left:6px solid #CA6F1E; padding:10px; border-radius:8px;">
⚡ <b>Pipeline</b> ile <b>AutoTokenizer</b> ve <b>AutoModel</b> arka planda otomatik olarak eşleştirilir.<br>
Sadece <b>görev adını</b> ve <b>modeli</b> belirtmeniz yeterli!
</div>

### 👩‍💻 <span style="color:#CA6F1E;">Pipeline</span> ile Duygu Analizi

In [ ]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis", model="distilbert-base-uncased")
print(nlp("Transformers çok güçlü!"))

**Açıklama:**
- Sadece görev adını ve modeli belirtmeniz yeterli.
- Model ve tokenizer otomatik yüklenir.
- Sonuç: Duygu analizi sonucu (pozitif/negatif) gelir.

<div style="background:#fadbd8; border-left:6px solid #C0392B; padding:10px; border-radius:8px;">
🎯 <b>Avantaj:</b> <b>Pipeline</b> ile <span style="color:#229954;">AutoTokenizer</span> ve <span style="color:#2980B9;">AutoModel</span> kullanımı <b>tek satıra</b> iner. Hızlı prototipleme ve test için idealdir!
</div>

> <b>Pipeline ile:</b> Metin sınıflandırma, özetleme, çeviri, soru-cevap gibi birçok NLP görevini kolayca yapabilirsiniz.

## 4. Transformer Mimarileri: GPT, BERT, T5

<div style="background:#f4fbf4; border-left:6px solid #229954; padding:10px; border-radius:8px;">
🔬 <b>Her mimarinin kendine özgü avantajları ve kullanım alanları vardır.</b>
</div>

| <span style="color:#2980B9;">Model</span>  | <span style="color:#CA6F1E;">Mimari</span>         | <span style="color:#229954;">Güçlü Yönler</span>           | <span style="color:#C0392B;">Kullanım Alanları</span>                 |
|--------|----------------|------------------------|-----------------------------------|
| <b>GPT</b>    | Decoder-only   | Uzun metin üretimi     | Yaratıcı yazım, sohbet botları    |
| <b>BERT</b>   | Encoder-only   | Anlam çıkarma, analiz  | Sınıflandırma, NER, Soru-Cevap    |
| <b>T5</b>     | Encoder-Decoder| Her şey text-to-text   | Çeviri, özetleme, çoklu görevler  |

<div style="background:#e8daef; border-left:6px solid #8E44AD; padding:10px; border-radius:8px;">
🧠 <b>Flashcard:</b> <b>GPT</b> üretkenlikte, <b>BERT</b> anlamada, <b>T5</b> ise çoklu görevlerde üstündür.
</div>

## 5. Cihaz ve Model Optimizasyonu

<div style="background:#f9e79f; border-left:6px solid #b7950b; padding:10px; border-radius:8px;">
⚙️ <b>Optimizasyon</b> ile <span style="color:#229954;">hız</span>, <span style="color:#C0392B;">verimlilik</span> ve <span style="color:#2980B9;">düşük maliyet</span> elde edilir.
</div>

### 👩‍💻 Temel Kod Örneği

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
with torch.no_grad():
    outputs = model(**{k: v.to(device) for k, v in inputs.items()})

**Açıklama:**
- Model ve veriler uygun cihaza taşınır.
- `torch.no_grad()` ile gereksiz hesaplamalar engellenir (daha hızlı ve az bellek kullanır).

<div style="background:#fadbd8; border-left:6px solid #C0392B; padding:10px; border-radius:8px;">
💡 <b>İpucu:</b> <b>Quantization</b> (8-bit, 4-bit) ve <b>batch processing</b> ile büyük modelleri bile düşük donanımda hızlıca çalıştırabilirsiniz!
</div>

> **Not:** Quantization ve batch processing ile ilgili ileri teknikler için Hugging Face ve PyTorch dökümantasyonuna bakabilirsiniz.

## 6. Performans Ölçümü ve Benchmarking

<div style="background:#f4f8fb; border-left:6px solid #2980B9; padding:10px; border-radius:8px;">
⏱️ <b>Performans ölçümü</b> ile <span style="color:#C0392B;">en hızlı ve verimli</span> pipeline'ı bulabilirsiniz.
</div>

### 👩‍💻 Basit Ölçüm Örneği

In [ ]:
import time
start = time.time()
_ = nlp(["Test cümlesi"] * 32)
print("Süre:", time.time() - start, "saniye")

**Açıklama:**
- 32 örnek için inference süresi ölçülür.
- Batch size ve cihaz değiştikçe bu süre değişir.

<div style="background:#f9e79f; border-left:6px solid #b7950b; padding:10px; border-radius:8px;">
📊 <b>Önemli:</b> <b>Inference süresi</b>, <b>memory kullanımı</b> ve <b>throughput</b> üretim ortamında kritik metriklerdir!
</div>

> **İpucu:** Daha detaylı ölçümler için `psutil`, `torch.cuda.memory_allocated()` gibi araçlar kullanılabilir.

## 7. En İyi Uygulamalar (Best Practices)

<div style="background:#f4fbf4; border-left:6px solid #229954; padding:10px; border-radius:8px;">
✅ <b>Her zaman</b> <b>torch.no_grad()</b> ile inference yapın, <b>cihazı</b> otomatik seçin ve <b>memory leak</b> riskine karşı temizlik yapın!
</div>

### 🔹 Performans Optimizasyonu
```python
# ✅ İyi
with torch.no_grad():
    outputs = model(**inputs)

# ❌ Kötü
outputs = model(**inputs)  # Gradient hesaplanır
```

### 🔹 Device Yönetimi
```python
# ✅ İyi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# ❌ Kötü
model = model.to("cuda")  # CUDA olmayabilir
```

### 🔹 Memory Yönetimi
```python
# ✅ İyi
del model
torch.cuda.empty_cache()
import gc; gc.collect()

# ❌ Kötü
# Memory leak'e sebep olabilir
```

<div style="background:#fadbd8; border-left:6px solid #C0392B; padding:10px; border-radius:8px;">
🟠 <b>Flashcard:</b> <b>Pipeline</b> ile <b>AutoTokenizer</b> ve <b>AutoModel</b> arka planda otomatik yönetilir. <b>Manuel kullanımda</b> ise cihaz ve memory yönetimi sizin sorumluluğunuzdadır!
</div>

## 8. Ek Kaynaklar

- <span style="color:#2980B9;">[Hugging Face Transformers Documentation](https://huggingface.co/docs/transformers/)</span>
- <span style="color:#229954;">[PyTorch Performance Tuning Guide](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html)</span>
- <span style="color:#C0392B;">[BERT Paper (Devlin et al., 2018)](https://arxiv.org/abs/1810.04805)</span>
- <span style="color:#CA6F1E;">[GPT Paper (Radford et al., 2018)](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)</span>
- <span style="color:#8E44AD;">[T5 Paper (Raffel et al., 2019)](https://arxiv.org/abs/1910.10683)</span>

---

<div align="center" style="font-size:1.2em; background:#f4f8fb; border-radius:8px; padding:16px; border:2px solid #CA6F1E;">
  <b>🌟 <span style="color:#CA6F1E;">Transformers ile optimize pipeline</span>, <span style="color:#229954;">üretim kalitesinde NLP'nin anahtarıdır!</span> 🌟</b>
</div>